In [102]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import os

# --- Configuration ---
# Set to True for single image debugging with visualizations, False for batch processing
DEBUG_MODE = False
# Specify the path to the single image for debugging mode
DEBUG_IMAGE_PATH = 'Training_Data/1M_1-4W/1M_1-4W_(3).jpg'

# Parameters for image processing
# Resistor detection parameters
BLUR_DIAMETER = 2
BLUR_SIGMA_COLOR = 40
BLUR_SIGMA_SPACE = 40
ADAPTIVE_BLOCK_SIZE = 151 # Must be odd
ADAPTIVE_C = 20
MORPH_KERNEL_SIZE = (5, 5)
MORPH_CLOSE_ITER = 6
MORPH_OPEN_ITER = 1
MIN_CONTOUR_AREA = 300
MIN_ASPECT_RATIO = 1.8
UPSCALE_FACTOR = 2
UPSCALE_INTERPOLATION = cv2.INTER_CUBIC
MIN_COLOR_STD_SUM = 30

# Conditional Morphology Heuristic
SATURATION_THRESHOLD_FOR_SIMPLE_BACKGROUND = 30
MORPH_KERNEL_SIZE_SIMPLE = (3, 3)
MORPH_CLOSE_ITER_SIMPLE = 5
MORPH_OPEN_ITER_SIMPLE = 5

# Resistor Enhancement Parameters
SHARPEN_KERNEL = np.array([
    [0, -1, 0],
    [-1, 5,-1],
    [0, -1, 0]
])
GAMMA_VALUE = 0.6
BILATERAL_D = 9
BILATERAL_SIGMA_COLOR = 75
BILATERAL_SIGMA_SPACE = 75
CLAHE_CLIP_LIMIT = 2.0
CLAHE_TILE_GRID_SIZE = (4, 4)

# Band Segmentation Parameters
BAND_MASK_PADDING_PERCENTAGE = 0.1
BAND_MORPH_KERNEL_SIZE = (3, 3)
BAND_MORPH_CLOSE_ITER = 3
BAND_MORPH_OPEN_ITER = 1

# Band Contour Filtering Parameters
MIN_BAND_CONTOUR_AREA = 40
MIN_BAND_ASPECT_RATIO = 1.6
MIN_BAND_HEIGHT = 5
MIN_BAND_WIDTH = 0.5
MIN_BAND_CONFIDENCE = 70

# HSV color ranges for resistor bands (H: 0-180, S: 0-255, V: 0-255)
COLOR_RANGES = {
    'black': [(np.array([0, 0, 0]), np.array([180, 255, 40]))],
    'brown': [(np.array([5, 140, 20]), np.array([25, 255, 70]))],
    'red': [
        (np.array([0, 120, 80]), np.array([8, 255, 255])),
        (np.array([170, 120, 80]), np.array([180, 255, 255]))
    ],
    'orange': [(np.array([8, 180, 140]), np.array([20, 255, 255]))],
    'yellow': [(np.array([20, 170, 100]), np.array([40, 255, 255]))],
    'green': [(np.array([40, 50, 30]), np.array([85, 255, 255]))],
    'blue': [(np.array([90, 100, 60]), np.array([130, 255, 255]))],
    'violet': [(np.array([115, 30, 30]), np.array([165, 255, 255]))],
    'grey': [(np.array([0, 0, 80]), np.array([180, 25, 170]))],
    'white': [(np.array([0, 0, 250]), np.array([180, 25, 255]))],
    'gold': [(np.array([15, 180, 50]), np.array([45, 245, 235]))],
    'silver': [(np.array([0, 0, 190]), np.array([180, 20, 245]))]
}

# Resistor Color Code Mapping
COLOR_DIGIT = {
    'black': 0, 'brown': 1, 'red': 2, 'orange': 3,
    'yellow': 4, 'green': 5, 'blue': 6, 'violet': 7,
    'grey': 8, 'white': 9
}

MULTIPLIER_MAP = {
    'black': 1, 'brown': 10, 'red': 100, 'orange': 1_000,
    'yellow': 10_000, 'green': 100_000, 'blue': 1_000_000,
    'gold': 0.1, 'silver': 0.01
}

TOLERANCE_COLORS = ['gold', 'silver']

# Helper function to display images (optional, for visualization during development)
def display_image(img, title="Image", cmap=None, figsize=(8, 6)):
    """Displays an image using matplotlib. Only shows if DEBUG_MODE is True."""
    if DEBUG_MODE:
        plt.figure(figsize=figsize)
        if cmap:
            plt.imshow(img, cmap=cmap)
        else:
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(title)
        plt.axis("off")
        plt.show()

def extract_resistor_region(original_img, resistor_rect):
    """Extracts the rotated rectangular region of the resistor using robust sorting."""
    center, size, angle = resistor_rect
    width, height = size

    # Standardize orientation so width is always the longer dimension
    if height > width:
        angle += 90
        width, height = height, width

    box_pts = cv2.boxPoints(resistor_rect)
    box_pts = np.float32(box_pts) # Needs to be float for getPerspectiveTransform

    if box_pts.shape[0] != 4:
        if DEBUG_MODE:
            print(f"Warning: minAreaRect did not return 4 points ({box_pts.shape[0]}), cannot extract region.")
        return None

    extracted_width = int(width)
    extracted_height = int(height)

    if extracted_width <= 0 or extracted_height <= 0:
        if DEBUG_MODE:
            print(f"Warning: Calculated extraction dimensions are invalid ({extracted_width}x{extracted_height}). Skipping extraction.")
        return None

    dst_pts = np.array([
        [0, 0],
        [extracted_width - 1, 0],
        [extracted_width - 1, extracted_height - 1],
        [0, extracted_height - 1]
    ], dtype="float32")

    # --- Robust point sorting logic (X then Y) ---
    # Sort the points based on their x-coordinates
    xsorted = box_pts[np.argsort(box_pts[:, 0]), :]

    # Grab the left-most and right-most points from the x-sorted points
    leftMost = xsorted[:2, :]
    rightMost = xsorted[2:, :]

    # Sort the left-most coordinates by their y-coordinate to get the top-left and bottom-left
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (tl, bl) = leftMost

    # Now, sort the right-most coordinates by their y-coordinate to get the top-right and bottom-right
    rightMost = rightMost[np.argsort(rightMost[:, 1]), :]
    (tr, br) = rightMost

    # Put the points in the order: top-left, top-right, bottom-right, bottom-left
    rect_points = np.array([tl, tr, br, bl], dtype="float32")
    # --- End of robust point sorting logic ---

    M = cv2.getPerspectiveTransform(rect_points, dst_pts)

    # Apply the perspective transform to the original image
    extracted_img = cv2.warpPerspective(original_img, M, (extracted_width, extracted_height))

    if extracted_img.size == 0:
        if DEBUG_MODE:
            print("Warning: Extracted resistor area is empty after warpPerspective!")
        return None

    display_image(extracted_img, "Extracted Resistor Region")

    return extracted_img


def check_color_variation(extracted_img, min_color_std_sum):
    """Calculates the color variation within the extracted region. Prints info if DEBUG_MODE is True."""
    if extracted_img is None or extracted_img.size == 0:
        return False

    std_dev_channels = np.std(extracted_img, axis=(0, 1))
    std_sum = np.sum(std_dev_channels)
    if DEBUG_MODE:
        print(f"Color Std Dev Sum on extracted region: {std_sum:.2f}")
    return std_sum >= min_color_std_sum

def upscale_image(img, upscale_factor, interpolation_method):
    """Upscales an image by a specified factor."""
    if img is None or img.size == 0 or upscale_factor == 1:
        return img

    new_width = int(img.shape[1] * upscale_factor)
    new_height = int(img.shape[0] * upscale_factor)

    if new_width <= 0 or new_height <= 0:
        return img

    upscaled_img = cv2.resize(img, (new_width, new_height), interpolation=interpolation_method)
    if DEBUG_MODE:
        print(f"Upscaled image size: {upscaled_img.shape[1]}x{upscaled_img.shape[0]}")
    return upscaled_img

def enhance_resistor_image(img, sharpen_kernel, gamma_value, bilateral_d, bilateral_sigma_color, bilateral_sigma_space, clahe_clip_limit, clahe_tile_grid_size):
    """Applies sharpening, gamma correction, and CLAHE (on S and V) for enhancement. Displays final enhanced image if DEBUG_MODE is True."""
    if img is None or img.size == 0:
        return img

    sharpened = cv2.filter2D(img, -1, sharpen_kernel)
    lookUpTable = np.array([((i / 255.0) ** gamma_value) * 255 for i in np.arange(0, 256)]).astype("uint8")
    brightened_sharpened = cv2.LUT(sharpened, lookUpTable)
    denoised_before_clahe = cv2.bilateralFilter(brightened_sharpened, d=bilateral_d, sigmaColor=bilateral_sigma_color, sigmaSpace=bilateral_sigma_space)

    hsv_img_for_clahe = cv2.cvtColor(denoised_before_clahe, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_img_for_clahe)
    clahe = cv2.createCLAHE(clipLimit=clahe_clip_limit, tileGridSize=clahe_tile_grid_size)
    v_clahe = clahe.apply(v)
    s_clahe = clahe.apply(s)
    enhanced_hsv = cv2.merge((h, s_clahe, v_clahe))
    vibrant_resistor = cv2.cvtColor(enhanced_hsv, cv2.COLOR_HSV2BGR)

    final_enhanced = cv2.bilateralFilter(vibrant_resistor, bilateral_d, bilateral_sigma_color, bilateral_sigma_space)

    display_image(final_enhanced, "Final Enhanced Resistor Image")
    return final_enhanced

def calculate_band_confidence(masked_region, hsv_image, ranges_list):
    """Calculates the percentage of pixels in the masked region matching color ranges."""
    if not ranges_list:
        return 0.0

    combined_mask = np.zeros_like(masked_region, dtype=np.uint8)

    for lower_hsv, upper_hsv in ranges_list:
        lower_bound = np.array(lower_hsv)
        upper_bound = np.array(upper_hsv)
        color_mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
        combined_mask = cv2.bitwise_or(combined_mask, color_mask)

    matched_pixels = np.sum(np.bitwise_and(masked_region, combined_mask) > 0)
    total_region_pixels = np.sum(masked_region > 0)

    if total_region_pixels == 0:
        return 0.0

    confidence = (matched_pixels / total_region_pixels) * 100
    return confidence

def segment_color_bands(enhanced_img, band_mask_padding_percentage, band_morph_kernel_size, band_morph_close_iter, band_morph_open_iter, min_band_contour_area, min_band_aspect_ratio, min_band_height, min_band_width, min_band_confidence, color_ranges):
    """Segments potential color bands using HSV ranges, morphology, and contour filtering. Prints/displays if DEBUG_MODE is True."""
    if enhanced_img is None or enhanced_img.size == 0:
        return []

    img_hsv = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2HSV)
    height, width = img_hsv.shape[:2]

    padding_w = int(band_mask_padding_percentage * width)
    padding_h = int(band_mask_padding_percentage * height)

    if padding_w >= width / 2 or padding_h >= height / 2:
        padding_w = int(width * 0.05)
        padding_h = int(height * 0.05)
        if padding_w <= 0 or padding_h <= 0: padding_w = padding_h = 1


    resistor_center_mask = np.zeros((height, width), dtype=np.uint8)
    cv2.rectangle(resistor_center_mask, (padding_w, padding_h), (width - padding_w, height - padding_h), 255, -1)
    display_image(resistor_center_mask, "Mask to remove edges for bands", cmap="gray")


    band_info = []
    band_morph_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, band_morph_kernel_size)

    if DEBUG_MODE:
        print("Starting color mask generation and band contour analysis...")

    for color_name, ranges_list in color_ranges.items():
        color_mask = np.zeros((height, width), dtype=np.uint8)

        for lower, upper in ranges_list:
            range_mask = cv2.inRange(img_hsv, lower, upper)
            color_mask = cv2.bitwise_or(color_mask, range_mask)

        masked = cv2.bitwise_and(color_mask, resistor_center_mask)

        masked_cleaned = cv2.morphologyEx(masked, cv2.MORPH_CLOSE, band_morph_kernel, iterations=band_morph_close_iter)
        masked_cleaned = cv2.morphologyEx(masked_cleaned, cv2.MORPH_OPEN, band_morph_kernel, iterations=band_morph_open_iter)

        display_image(masked_cleaned, f"Cleaned Mask for {color_name} (center only)", cmap="gray")

        contours, _ = cv2.findContours(masked_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            area = cv2.contourArea(cnt)

            if (area > min_band_contour_area and
                h > min_band_height and
                w > min_band_width):

                aspect_ratio = h / float(w) if w != 0 else 0
                if aspect_ratio < min_band_aspect_ratio:
                    continue

                contour_mask = np.zeros_like(masked_cleaned, dtype=np.uint8)
                cv2.drawContours(contour_mask, [cnt], -1, 255, -1)

                confidence = calculate_band_confidence(contour_mask, img_hsv, ranges_list)

                if confidence > min_band_confidence:
                    band_center_x = x + w // 2
                    if DEBUG_MODE:
                        print(f"Accepted band: {color_name} at x={band_center_x} with confidence {confidence:.2f}% (Area={area:.1f}, AR={aspect_ratio:.2f})")
                    band_info.append((band_center_x, color_name))
                elif DEBUG_MODE:
                    print(f"Rejected band: {color_name} at x={x} due to low confidence {confidence:.2f}%")


    if DEBUG_MODE:
        print("Finished band contour analysis.")
    return band_info

def calculate_resistor_value(bands, color_digit_map, multiplier_map):
    """Calculates the resistor value based on the detected and sorted bands."""
    value = None
    num_bands = len(bands)

    try:
        if num_bands >= 4:
            if num_bands >= 3:
                digit1 = color_digit_map.get(bands[0], None)
                digit2 = color_digit_map.get(bands[1], None)

                if num_bands == 4:
                    multiplier = multiplier_map.get(bands[2], None)
                    if digit1 is not None and digit2 is not None and multiplier is not None:
                        value = (10 * digit1 + digit2) * multiplier

                elif num_bands == 5:
                    if num_bands >= 4:
                        digit3 = color_digit_map.get(bands[2], None)
                        multiplier = multiplier_map.get(bands[3], None)
                        if digit1 is not None and digit2 is not None and digit3 is not None and multiplier is not None:
                            value = (100 * digit1 + 10 * digit2 + digit3) * multiplier
        else:
            pass # Not enough bands to calculate value

    except Exception as e:
        if DEBUG_MODE:
            print(f"Error calculating resistor value: {e}")
        value = None

    return value

def read_resistor_code(band_info, tolerance_colors, color_digit_map, multiplier_map):
    """Processes band information, sorts bands, and calculates the resistor value. Prints info if DEBUG_MODE is True."""
    if not band_info:
        if DEBUG_MODE:
            print("Did not detect any valid bands.")
        return [], None

    band_info.sort(key=lambda x: x[0])
    band_colors = [color for x, color in band_info]

    if DEBUG_MODE:
        print("Sorted band colors (before potential flip/adding) are:", band_colors)

    if band_colors and band_colors[0] in tolerance_colors:
        if DEBUG_MODE:
            print("Detected a tolerance band potentially first, attempting to flip the band order.")
        band_colors.reverse()
        if DEBUG_MODE:
            print("Band colors after flip attempt:", band_colors)


    if len(band_colors) == 3:
        has_tolerance_color_in_first_three = any(color in tolerance_colors for color in band_colors)
        if not has_tolerance_color_in_first_three:
            if DEBUG_MODE:
                print("Detected 3 bands, none are typical tolerance colors. Assuming a missing Gold tolerance band.")
            band_colors.append('gold')
            if DEBUG_MODE:
                print("Band colors after assuming gold tolerance:", band_colors)


    num_bands = len(band_colors)
    if num_bands < 4 or num_bands > 5:
        if DEBUG_MODE:
            print(f"Warning: Detected {num_bands} bands. Calculation logic currently supports 4 or 5 bands.")
        return band_colors, None

    calculated_value = calculate_resistor_value(band_colors, color_digit_map, multiplier_map)

    if DEBUG_MODE:
        if calculated_value is not None:
            print(f"Calculated resistor value: {calculated_value} ohms")
        else:
            print("Could not calculate the resistor value with the detected bands (check band colors/order).")


    return band_colors, calculated_value

def process_single_image(image_path):
    """Processes a single image to detect a resistor and read its code. Includes debug prints/displays if DEBUG_MODE is True."""
    if DEBUG_MODE:
        print(f"Processing image: {image_path}")
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not load the image at {image_path}")
        return None, None

    display_image(img, "Original Image")

    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img_hsv)

    # --- Determine background complexity ---
    average_saturation = np.mean(s)
    if DEBUG_MODE:
        print(f"Average Saturation: {average_saturation:.2f}")


    # --- Conditional Thresholding based on Background ---
    # Simple background has low saturation
    if average_saturation < SATURATION_THRESHOLD_FOR_SIMPLE_BACKGROUND:
        if DEBUG_MODE:
            print(f"Detected simple background (Avg Sat: {average_saturation:.2f}) - using Adaptive Threshold.")
        processed_mask = cv2.adaptiveThreshold(v, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, ADAPTIVE_BLOCK_SIZE, ADAPTIVE_C)
        display_image(processed_mask, "Adaptive Threshold (Value Channel)", cmap='gray')
    else: # Complex background has higher saturation
        if DEBUG_MODE:
            print(f"Detected complex background (Avg Sat: {average_saturation:.2f}) - using Value threshold.")
        processed_mask = cv2.threshold(v, 200, 255, cv2.THRESH_BINARY)[1]
        display_image(processed_mask, "Standard Value Threshold Mask", cmap='gray')


    # --- Apply Morphology based on Background ---
    morph_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, MORPH_KERNEL_SIZE)

    if average_saturation < SATURATION_THRESHOLD_FOR_SIMPLE_BACKGROUND: # Simple background, use weaker morphology
        open_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, MORPH_KERNEL_SIZE_SIMPLE)
        closed_mask = cv2.morphologyEx(processed_mask, cv2.MORPH_CLOSE, morph_kernel, iterations=MORPH_CLOSE_ITER_SIMPLE)
        final_mask_for_contours = cv2.morphologyEx(closed_mask, cv2.MORPH_OPEN, open_kernel, iterations=MORPH_OPEN_ITER_SIMPLE)
    else: # Complex background, use stronger morphology
        open_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, MORPH_KERNEL_SIZE)
        open_mask = cv2.morphologyEx(processed_mask, cv2.MORPH_OPEN, open_kernel, iterations=MORPH_OPEN_ITER)
        final_mask_for_contours = cv2.morphologyEx(open_mask, cv2.MORPH_CLOSE, morph_kernel, iterations=MORPH_CLOSE_ITER)


    display_image(final_mask_for_contours, f"Morphologically Processed Mask (Avg Sat: {average_saturation:.2f})", cmap='gray')

    # Contour Detection
    contours, hierarchy = cv2.findContours(final_mask_for_contours, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if DEBUG_MODE:
        print(f"Found {len(contours)} initial contours")

    # Filter the detected contours by area and aspect ratio
    potential_resistors = []
    img_for_contours = img.copy() if DEBUG_MODE else None


    for cnt in contours:
        area = cv2.contourArea(cnt)

        if area < MIN_CONTOUR_AREA:
            continue

        rect = cv2.minAreaRect(cnt)
        (x, y), (width, height), angle = rect

        width = max(1, width)
        height = max(1, height)

        aspect_ratio = max(width / height, height / width)

        if aspect_ratio < MIN_ASPECT_RATIO:
            continue

        potential_resistors.append({'contour': cnt, 'rect': rect, 'area': area})

        # Optionally draw the potential contours before color filtering (Draw on a copy of the original)
        if DEBUG_MODE and img_for_contours is not None:
            box = cv2.boxPoints(rect)
            box = np.intp(box)
            cv2.drawContours(img_for_contours, [box], 0, (0, 255, 0), 2) # Draw green box

    if DEBUG_MODE:
        print(f"Found {len(potential_resistors)} potential contours after Area and Aspect Ratio filtering.")
        if img_for_contours is not None:
             display_image(img_for_contours, "Potential Contours (Area & Aspect Ratio filtered)")


    # Select the best candidate and Extract the resistor region
    best_resistor_data = None
    extracted_resistor_img = None

    if not potential_resistors:
        if DEBUG_MODE:
            print("Did not find any potential resistor contours based on initial filtering.")
        return None, None

    potential_resistors.sort(key=lambda item: item['area'], reverse=True)

    if DEBUG_MODE:
        print("Checking potential resistor candidates for sufficient color variation after extraction...")


    for candidate in potential_resistors:
        # Call the extract_resistor_region function with the robust sorting logic
        current_extracted_img = extract_resistor_region(img, candidate['rect'])

        if current_extracted_img is not None and check_color_variation(current_extracted_img, MIN_COLOR_STD_SUM):
            if DEBUG_MODE:
                print(f"Candidate passed color variation check (Area: {candidate['area']:.2f}).")
            best_resistor_data = candidate
            extracted_resistor_img = current_extracted_img
            # Optionally draw the selected contour on a copy of the original image here
            if DEBUG_MODE:
                img_with_best_contour = img.copy()
                box = cv2.boxPoints(candidate['rect'])
                box = np.intp(box)
                cv2.drawContours(img_with_best_contour, [box], 0, (0, 255, 0), 2) # Draw a green box
                display_image(img_with_best_contour, "Selected Resistor Contour (after color validation)")
            break

    if extracted_resistor_img is None or extracted_resistor_img.size == 0:
        if DEBUG_MODE:
            print("Did not detect any suitable resistor contour after extraction and color variation filtering.")
        return None, None

    # Upscale the extracted image
    upscaled_resistor_img = upscale_image(extracted_resistor_img, UPSCALE_FACTOR, UPSCALE_INTERPOLATION)

    if DEBUG_MODE:
        print("Proceeding with enhancement and band segmentation...")

    enhanced_resistor = enhance_resistor_image(upscaled_resistor_img, SHARPEN_KERNEL, GAMMA_VALUE, BILATERAL_D, BILATERAL_SIGMA_COLOR, BILATERAL_SIGMA_SPACE, CLAHE_CLIP_LIMIT, CLAHE_TILE_GRID_SIZE)

    band_info = segment_color_bands(enhanced_resistor, BAND_MASK_PADDING_PERCENTAGE, BAND_MORPH_KERNEL_SIZE, BAND_MORPH_CLOSE_ITER, BAND_MORPH_OPEN_ITER, MIN_BAND_CONTOUR_AREA, MIN_BAND_ASPECT_RATIO, MIN_BAND_HEIGHT, MIN_BAND_WIDTH, MIN_BAND_CONFIDENCE, COLOR_RANGES)

    detected_bands, calculated_value = read_resistor_code(band_info, TOLERANCE_COLORS, COLOR_DIGIT, MULTIPLIER_MAP)

    return detected_bands, calculated_value

# --- Main Execution ---
if __name__ == "__main__":
    # Define the directory containing the images
    image_directory = 'Training_Data/1M_1-4W/' # Path to the image directory

    results = {}

    print("--- Resistor Reader Script Starting ---")

    if not os.path.isdir(image_directory):
        print(f"Error: Directory not found at {image_directory}")
    else:
        if DEBUG_MODE:
            # Single image debugging mode
            if not os.path.exists(DEBUG_IMAGE_PATH):
                 print(f"Error: Debug image not found at {DEBUG_IMAGE_PATH}")
            else:
                 bands, value = process_single_image(DEBUG_IMAGE_PATH)
                 results[DEBUG_IMAGE_PATH] = {'bands': bands, 'value': value}
        else:
            # Batch processing mode
            image_files = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            if not image_files:
                print(f"No image files found in {image_directory}")
            else:
                 for img_path in image_files:
                     bands, value = process_single_image(img_path)
                     results[img_path] = {'bands': bands, 'value': value}

    # Print final results for all images processed
    print("\n--- Processing Results ---")
    if not results:
        print("No images were processed.")
    else:
        for img_path, result in results.items():
            print(f"\nImage: {img_path}")
            if result['bands'] is not None:
                print(f"Detected Bands: {result['bands']}")
                if result['value'] is not None:
                    print(f"Calculated Value: {result['value']} ohms")
                else:
                    print("Could not calculate resistor value.")
            else:
                print("No resistor detected or processed.")

--- Resistor Reader Script Starting ---

--- Processing Results ---

Image: Training_Data/1M_1-4W/1M_1-4W_(1).jpg
Detected Bands: ['green', 'black', 'brown', 'gold']
Calculated Value: 500 ohms

Image: Training_Data/1M_1-4W/1M_1-4W_(10).jpg
No resistor detected or processed.

Image: Training_Data/1M_1-4W/1M_1-4W_(11).jpg
Detected Bands: ['blue']
Could not calculate resistor value.

Image: Training_Data/1M_1-4W/1M_1-4W_(12).jpg
No resistor detected or processed.

Image: Training_Data/1M_1-4W/1M_1-4W_(13).jpg
Detected Bands: ['brown', 'orange']
Could not calculate resistor value.

Image: Training_Data/1M_1-4W/1M_1-4W_(14).jpg
Detected Bands: ['orange', 'orange']
Could not calculate resistor value.

Image: Training_Data/1M_1-4W/1M_1-4W_(15).jpg
Detected Bands: ['black', 'grey']
Could not calculate resistor value.

Image: Training_Data/1M_1-4W/1M_1-4W_(16).jpg
No resistor detected or processed.

Image: Training_Data/1M_1-4W/1M_1-4W_(17).jpg
No resistor detected or processed.

Image: Trainin